[View in Colaboratory](https://colab.research.google.com/github/ErikPartridge/machine-translation/blob/master/ENFR.ipynb)

In [0]:
!wget http://www.statmt.org/europarl/v7/fr-en.tgz

--2018-07-05 14:42:56--  http://www.statmt.org/europarl/v7/fr-en.tgz
Resolving www.statmt.org (www.statmt.org)... 129.215.197.184
Connecting to www.statmt.org (www.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 202718517 (193M) [application/x-gzip]
Saving to: ‘fr-en.tgz.2’

fr-en.tgz.2          10%[=>                  ]  20.66M   415KB/s    eta 7m 11s 

fr-en.tgz.2          37%[======>             ]  73.43M   451KB/s    eta 4m 58s 

In [2]:
!tar -xvf fr-en.tgz

europarl-v7.fr-en.en
europarl-v7.fr-en.fr


In [6]:
!sed -i '/^</ d' europarl-v7.fr-en.en 
!sed -i'/^</ d' europarl-v7.fr-en.fr
!sed -i '/^$/d' europarl-v7.fr-en.fr
!sed -i '/^$/d' europarl-v7.fr-en.en

sed: no input files


In [7]:
!head europarl-v7.fr-en.en

Resumption of the session
I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.
You have requested a debate on this subject in the course of the next few days, during this part-session.
In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.
Please rise, then, for this minute' s silence.
(The House rose and observed a minute' s silence)
Madam President, on a point of order.
You will be aware from the press and television that there have been a number of bomb explosions and

In [8]:
!head europarl-v7.fr-en.fr

Reprise de la session
Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances.
Comme vous avez pu le constater, le grand "bogue de l'an 2000" ne s'est pas produit. En revanche, les citoyens d'un certain nombre de nos pays ont été victimes de catastrophes naturelles qui ont vraiment été terribles.
Vous avez souhaité un débat à ce sujet dans les prochains jours, au cours de cette période de session.
En attendant, je souhaiterais, comme un certain nombre de collègues me l'ont demandé, que nous observions une minute de silence pour toutes les victimes, des tempêtes notamment, dans les différents pays de l'Union européenne qui ont été touchés.
Je vous invite à vous lever pour cette minute de silence.
(Le Parlement, debout, observe une minute de silence)
Madame la Présidente, c'est une motion de procédure.
Vous avez probablement appris par la presse et p

In [9]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')


with open ("europarl-v7.fr-en.fr", "r") as frf:
  fr = frf.readlines()
    
with open ("europarl-v7.fr-en.en", "r") as enf:
  en = enf.readlines()

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence

frt = []
for line in fr:
  frt.append(one_hot(line, 12000))

ent = []
for line in en:
  ent.append(one_hot(line, 12000))

In [0]:
import numpy as np

frv = []
for line in frt:
  tmp = []
  for num in line:
    l = np.zeros(12000)
    l[num] = 1
    tmp.append(l)
  frv.append(np.array(tmp))
frv = np.array(frv)

envr = []
for line in ent:
  tmp = []
  for num in line:
    l = np.zeros(12000)
    l[num] = 1
    tmp.append(l)
  envr.append(np.array(tmp))
envr = np.array(envr)

[4972, 7588, 5340, 4254, 2072, 11485, 5336, 5833, 8929, 4598, 7608, 3589, 9305, 4003, 5343, 10759, 5681, 9903, 4972, 7397, 5462, 8537, 11187, 5014, 1326, 3875, 601, 7397, 11718, 9394, 3092, 8163, 8209]


['You',
 'have',
 'requested',
 'a',
 'debate',
 'on',
 'this',
 'subject',
 'in',
 'the',
 'course',
 'of',
 'the',
 'next',
 'few',
 'days',
 'during',
 'this',
 'part-session']